# Task 1


$f(x_1, x_2) = (x_1 + x_2)^2$

$x_1, x_2$ ~ $U[-1,1] \wedge x_1 = x_2$ (full dependency)


$g_{PD}^1(z) = E_{X_2}[(z + x_2)^2] = E_{X_2}[(z + x_2)^2] = E_{X_2}[z^2] + E_{X_2}[2z \cdot x_2] + E_{X_2}[x_2^2] = z^2 + 2zE_{X_2}[x_2] + E_{X_2}[x_2^2] = z^2 + 2z \cdot 0 +  \int_{-1}^{1} \frac{x^2}{(1-(-1))} \,dx   = z^2 + \frac{1}{3} $


# Report

The goal of this homework was to get familiar with Ceteris Paribus (CP) and Partial Dependence profiles (PDP).

In this notebook I used 2 randomly selected test samples on heart attack dataset.

Firstly I trained random forrest and and explained afore mentioned samples with CP using dalex. I created three plots of non-binary features: age, resting blood pressure (trtbps), cholesterol (chol). The results are presented below.

![CP Random Forrest](https://github.com/edsuch21/eXplainableMachineLearning-2023/blob/main/Homeworks/HW4/EdwardSucharda/cp_rf.png?raw=true)

The results are different both sampls. Not only the mean values are very different but the shape of the curve is not the same. The plots are pretty flat. For the whole range of these features the output changes only about 5 pp. This shows that values of the rest of the features dominates impact on the output. 

Then I explained the same model with PDP and generated plots for the same features as explained in CP. The results are presented below.

![PDP Random Forrest](https://github.com/edsuch21/eXplainableMachineLearning-2023/blob/main/Homeworks/HW4/EdwardSucharda/pdp_rf.png?raw=true)

Amplitude did not change much but this plots show better average output for every sample in the dataset. If the data is sampled from sensible distribution then we get expected impact age on the output.

I also tested PDP on SVM model:

![PDP SVM](https://github.com/edsuch21/eXplainableMachineLearning-2023/blob/main/Homeworks/HW4/EdwardSucharda/pdp_svm.png?raw=true)

and Linear Regression:

 ![PDP Linear Regression](https://github.com/edsuch21/eXplainableMachineLearning-2023/blob/main/Homeworks/HW4/EdwardSucharda/pdp_lr.png?raw=true)

The plots for those two methods are much smoother than on Random Forrest. This is caused because of tree-based ensemble model characteristic that small change of some feature may rapidly change the output if the condition on the node has opposite result. For both methods the explanation is almost identical except the fact the linear regresion has linear plot and SVM is a little bit curvy. The plots does not look alike to plot for random forrest.

# Appendix

In [1]:
pip install dalex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import dalex as dx
import random

In [3]:
hearts_df = pd.read_csv("heart.csv")

In [4]:
hearts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [5]:
for i in range(4):
  hearts_df[f"cp{i}"] = (hearts_df["cp"] == i) * 1
for i in range(3):
  hearts_df[f"restecg{i}"] = (hearts_df["restecg"] == i) * 1

In [6]:
y_heart = hearts_df["output"]
x_heart = hearts_df.drop(columns=["cp", "restecg", "output"])

In [7]:
x_heart.describe()

,age,sex,trtbps,chol,fbs,thalachh,exng,oldpeak,slp,caa,thall,cp0,cp1,cp2,cp3,restecg0,restecg1,restecg2
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,131.623762,246.264026,0.148515,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.471947,0.165017,0.287129,0.075908,0.485149,0.501650,0.013201
std,9.082101,0.466011,17.538143,51.830751,0.356198,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.500038,0.371809,0.453171,0.265288,0.500606,0.500824,0.114325
min,29.000000,0.000000,94.000000,126.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,120.000000,211.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,55.000000,1.000000,130.000000,240.000000,0.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,61.000000,1.000000,140.000000,274.500000,0.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,77.000000,1.000000,200.000000,564.000000,1.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
for column in x_heart.columns:
  x_heart[column] = (x_heart[column] - x_heart[column].mean()) / x_heart[column].std()

In [9]:
X_train_heart, X_test_heart, y_train_heart, y_test_heart = train_test_split(x_heart, y_heart, test_size=0.33, random_state=446519)

In [10]:
rf_h = RandomForestClassifier(n_estimators = 300, max_depth=5, random_state=446519)
rf_h.fit(X_train_heart, y_train_heart)

RandomForestClassifier(max_depth=5, n_estimators=300, random_state=446519)

In [11]:
random.seed(446519)

idx1 = random.randint(0, X_test_heart.shape[0] - 1)
idx2 = random.randint(0, X_test_heart.shape[0] - 1)
indices = [idx1, idx2]

In [12]:
sample1 = X_test_heart.iloc[[idx1]]
sample2 = X_test_heart.iloc[[idx2]]

In [13]:
print(f"Probailty of heart attack in first sample equals: {rf_h.predict_proba(sample1)[0][1]*100}%")
print(f"Probailty of heart attack in second sample equals: {rf_h.predict_proba(sample2)[0][1]*100}%")

Probailty of heart attack in first sample equals: 55.38228180752807%
Probailty of heart attack in second sample equals: 23.03908323870259%


In [14]:
model_predicting_method = lambda m, d: m.predict_proba(d)[:, 1]

explainer_dx_rf_a = dx.Explainer(rf_h, X_test_heart, y_test_heart, predict_function=model_predicting_method, label="RF_heart_attack")

Preparation of a new explainer is initiated

  -> data              : 100 rows 18 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 100 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : RF_heart_attack
  -> predict function  : <function <lambda> at 0x7f8f3029a7a0> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0096, mean = 0.517, max = 0.972
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.866, mean = 0.0334, max = 0.948
  -> model_info        : package sklearn

A new explainer has been created!


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [15]:
explainer_dx_rf_a.model_performance()

,recall,precision,f1,accuracy,auc
RF_heart_attack,0.854545,0.854545,0.854545,0.84,0.896566


In [16]:
cp = explainer_dx_rf_a.predict_profile(X_test_heart.iloc[[idx1, idx2]], type='ceteris_paribus', y=y_test_heart.iloc[[idx1, idx2]])

Calculating ceteris paribus: 100%|██████████| 18/18 [00:00<00:00, 20.54it/s]


In [17]:
cp.plot(variables=["age", "trtbps", "chol"])

In [18]:
pdp = explainer_dx_rf_a.model_profile(random_state=446519)


Calculating ceteris paribus: 100%|██████████| 18/18 [00:04<00:00,  4.44it/s]


In [19]:
pdp.plot(variables=["age", "trtbps", "chol"])

In [20]:
from sklearn.svm import SVC
svc = SVC(random_state=446519, probability=True)
svc.fit(X_train_heart, y_train_heart)

SVC(probability=True, random_state=446519)

In [21]:
explainer_dx_svm = dx.Explainer(svc, X_test_heart, y_test_heart, predict_function=model_predicting_method, label="SVM_heart_attack")

Preparation of a new explainer is initiated

  -> data              : 100 rows 18 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 100 values
  -> model_class       : sklearn.svm._classes.SVC (default)
  -> label             : SVM_heart_attack
  -> predict function  : <function <lambda> at 0x7f8f3029a7a0> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.00806, mean = 0.515, max = 0.97
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.898, mean = 0.035, max = 0.984
  -> model_info        : package sklearn

A new explainer has been created!


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but SVC was fitted with feature names



In [22]:
pdp_svm = explainer_dx_svm.model_profile(random_state=446519)

Calculating ceteris paribus: 100%|██████████| 18/18 [00:02<00:00,  8.89it/s]


In [23]:
pdp_svm.plot(variables=["age", "trtbps", "chol"])

In [24]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_heart, y_train_heart)

LogisticRegression()

In [25]:
explainer_dx_lr = dx.Explainer(lr, X_test_heart, y_test_heart, predict_function=model_predicting_method, label="SVM_heart_attack")

Preparation of a new explainer is initiated

  -> data              : 100 rows 18 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 100 values
  -> model_class       : sklearn.linear_model._logistic.LogisticRegression (default)
  -> label             : SVM_heart_attack
  -> predict function  : <function <lambda> at 0x7f8f3029a7a0> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.000599, mean = 0.522, max = 0.997
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.954, mean = 0.028, max = 0.973
  -> model_info        : package sklearn

A new explainer has been created!


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names



In [26]:
pdp_lr = explainer_dx_lr.model_profile(random_state=446519)

Calculating ceteris paribus: 100%|██████████| 18/18 [00:00<00:00, 70.36it/s]


In [27]:
pdp_lr.plot(variables=["age", "trtbps", "chol"])